In [24]:
import pandas as pd
import numpy as np
import PIL.Image as img
import os

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
covid = "/content/drive/MyDrive/SARSCOVID/COVID"
nonCovid = "/content/drive/MyDrive/SARSCOVID/non-COVID"
Try = "/content/drive/MyDrive/SARSCOVID/Try"

In [42]:
def file(path):
  return [os.path.join(path, f) for f in os.listdir(path)]

def converData(folderName, className):
  images = file(folderName)
  imagesClass = []
  for image in images:
    imageRead = img.open(image).convert("L")
    imageResizing = imageRead.resize((28, 28))
    imageConvert = np.array(imageResizing).flatten()
    if className == "covid":
      datas = np.append(imageConvert, [0])
    elif className == "nonCovid":
      datas = np.append(imageConvert, [1])
    else:
      continue
    imagesClass.append(datas)
  return imagesClass


In [28]:
covidData = converData(covid, "covid")
covidDataDF = pd.DataFrame(covidData)
nonCovidData = converData(nonCovid, "nonCovid")
nonCovidDataDF = pd.DataFrame(nonCovidData)
allData = pd.concat([covidDataDF, nonCovidDataDF])

In [29]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

entry = np.array(allData)[:, :784]
quit = np.array(allData)[:, 784]

entryTrain, entryTest, quitTrain, quitTest = train_test_split(entry, quit, test_size = 0.2, random_state = 42)

In [30]:
model = DecisionTreeClassifier()

In [31]:
clf = model.fit(entryTrain, quitTrain)

In [32]:
quitPredict = clf.predict(entryTest)

In [38]:
print("Accuracy", metrics.accuracy_score(quitTest, quitPredict) * 100)

Accuracy 84.10462776659959


# Dışardan veri girip sonuç alma

In [68]:
def read_and_predict_image(file_path):
    imageRead1 = img.open(file_path).convert("L")
    imageResizing1= imageRead1.resize((28, 28))
    imageConvert1 = np.array(imageResizing1).flatten()
    imageConvert1 = np.reshape(imageConvert1, (1, -1))
    return [clf.predict(imageConvert1)]

file_path = "/content/drive/MyDrive/SARSCOVID/Try/testData.png"
prediction = read_and_predict_image(file_path)
print(prediction)
if prediction == [0]:
  print("Covid")
else:
  print("No Covid")





[array([0])]
Covid


In [ ]:
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
wrongPositive, correctPositive, threshold = metrics.roc_curve(quitTest, quitPredict)
rocAuc = metrics.auc(wrongPositive, correctPositive)
plt.title('ROC curve')
plt.plot(wrongPositive, correctPositive, 'b', label = 'AUC = %0.2f' % rocAuc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('Correct positive rate')
plt.xlabel('Wrong positive rate')
plt.show()